In [1]:
import cvxpy as cp
import pandas as pd
import numpy as np
import cplex
import scipy.stats as stats

In [9]:
np.cov(market_[0, :], hc_[0, :])[0, 1] / 

0.5292662696741898

In [6]:
x_n = 2
n = 20000
gamma = 6
rho = 0.2

np.random.seed(x_n)

market_ = np.random.randn(n, 30)
brownian_ = np.random.randn(n, 30)
hc_ = rho * market_ + np.sqrt(1 - rho ** 2) * brownian_

risky_ret = cp.exp(0.12+(0.5*(0.33**2))+0.33*market_[:, 0])
rf_ret = cp.exp(0.04*np.ones(market_[:, 0].shape))

h_ret = 5000*np.cumprod(np.exp(0.04+0.03*(rho * market_ + np.sqrt(1 - rho ** 2) * brownian_))[:, 1:], axis=1)
dc_rate = cp.exp((0.04 + 0.04+) * (np.array(range(h_ret.shape[1])) + 1) * (-1))

x = cp.Variable(2)

W_t = (1000+5000*0.3)*(cp.bmat([risky_ret, rf_ret]).T @ x)
H_t = h_ret @ dc_rate

obj = cp.sum((W_t+H_t)**(1-gamma) / (1-gamma))
cons = [cp.sum(x) == 1, x >= 0]#, W_t == (1000+5000*0.3)*(cp.bmat([risky_ret, rf_ret]).T @ x)]

prob = cp.Problem(cp.Maximize(obj), cons)
prob.solve(cp.CVXOPT, verbose=True)
print(np.round(x.value, 3)*100)

SolverError: Either candidate conic solvers (['CVXOPT']) do not support the cones output by the problem (SOC, ExpCone), or there are not enough constraints in the problem.

In [2]:
from scipy.optimize import minimize

In [27]:
n = 20000
gamma = 10
rho = 0.2

market_ = np.random.randn(n, 30)
brownian_ = np.random.randn(n, 30)
hc_ = rho * market_ + np.sqrt(1 - rho ** 2) * brownian_

risky_ret = np.exp(0.12+(0.5*(0.33**2))+0.33*market_[:, 0])
rf_ret = np.exp(0.04*np.ones(market_[:, 0].shape))

h_ret = 5000*np.cumprod(np.exp(0.04+0.03*(rho * market_ + np.sqrt(1 - rho ** 2) * brownian_)), axis=1)
dc_rate = np.exp((0.04 + 0.04) * (np.array(range(30)) + 1) * (-1))

H_t = h_ret @ dc_rate

obj = lambda x: -np.mean(((1000+5000*0.3)*(np.array([risky_ret, rf_ret]).T @ x)+H_t)**(1-gamma) / (1-gamma))
#cons = ({'type': 'eq', 'fun': lambda x: ((1000+5000*0.3)*(np.mat([risky_ret, rf_ret]).T @ x) - W_t)})
cons = ({'type': 'ineq', 'fun': lambda x: -1.0 + np.sum(x)}, 
        {'type': 'ineq', 'fun': lambda x: x}, 
        {'type': 'eq', 'fun': lambda x: (1000+5000*0.3)*(np.array([risky_ret, rf_ret]).T @ x)})

optimized = minimize(obj, x0, method="trust-constr", constraints=cons)
if not optimized.success: raise BaseException(optimized.message)
print(optimized.x)

ValueError: expected square matrix

In [ ]:



cons = [cp.sum(x) == 1, x >= 0, W_t == (1000+5000*0.3)*(cp.bmat([risky_ret, rf_ret]).T @ x)]

prob = cp.Problem(cp.Maximize(obj), cons)
prob.solve(cp.ECOS, verbose=True)
print(np.round(x.value, 3)*100)

In [ ]:
cons = [x <= 1, x >= 0]
prob = cp.Problem(cp.Maximize(obj), cons)
prob.solve(cp.CPLEX)
print(np.round(x.value, 3)*100)

In [ ]:
size = 20000
period = 30
gamma = 2
rho = 0.2

ic = 5000
fa = 1000

hc_mu = 0.04
hcs = 0.01

r_s = 0.12
r_m = 0.33
rf = 0.04

np.random.seed(period)

market_ = stats.norm.rvs(size=(size, period), loc=0, scale=1)
brownian_ = stats.norm.rvs(size=(size, period), loc=0, scale=1)
hc_ = rho * market_ + np.sqrt(1 - rho ** 2) * brownian_

In [ ]:
labor_income = gen_labor_income(ic, hc_mu, hcs, .000002, hc_, rf, size, period)
labor_income

In [ ]:
discount_rate = get_discount_rate(rf, .000002, period)
discount_rate

In [ ]:
risky_ret, rf_ret = gen_finasset_ret(r_s, r_m, market_, rf, size, period)
risky_ret

In [ ]:
np.concatenate([[risky_ret[:, 0]], [rf_ret[:, 0]]]).T @ np.array([.5, .5])

In [ ]:
5000 * ((rf_ret[:, 1] * 0.3) + (risky_ret[:, 1] * 0.7))

In [ ]:
from scipy.optimize import minimize

In [ ]:


x0 = np.array([0.5, 0.5])
cons = ({'type': 'ineq', 'fun': lambda weights: -1.0 + np.sum(weights)},
        {'type': 'ineq', 'fun': lambda weights: weights})

optimized = minimize(crra_utility, x0, (gamma, risky_ret, rf_ret, ic, fa, labor_income @ discount_rate),
                   method="COBYLA", constraints=cons)
if not optimized.success: raise BaseException(optimized.message)
print(optimized.x)

In [ ]:
size = 20000
period = 30
gamma = 10
rho = 0.2

ic = 5000
fa = 1000

hc_mu = 0.04
hcs = 0.12
eta = 0.0000002

r_m = 0.12
r_s = 0.20
rf = 0.04
rf_s = 0

In [ ]:
size = 200
period = 35
gamma = 2
rho = 0.2

ic = 5000
fa = 1000

r_m = 0.12
r_s = 0.25
rf = 0.04
rf_s = 0

hc_mu = 0.02
hcs = 0.01

In [ ]:
np.random.seed(period)

market_ = stats.norm.rvs(size=(size, period), loc=0, scale=1)
brownian_ = stats.norm.rvs(size=(size, period), loc=0, scale=1)
hc_ = rho * market_ + np.sqrt(1 - rho ** 2) * brownian_

labor_income = gen_labor_income(ic, hc_mu, hcs, .000002, hc_, rf, size, period)
discount_rate = get_discount_rate(rf, .000002, period)

W_ret = gen_finasset_ret(r_m, r_s, market_[:, 0], rf)

In [ ]:
gamma_list = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]
for g in gamma_list:
    x = cp.Variable(2)
    
    if g == 1:
        obj = -cp.sum(cp.log((fa * gen_finasset_ret(r_m, r_s, market_[:, 0], rf) @ x) + (labor_income @ discount_rate).values))
    else:
        obj = -cp.sum((((fa * gen_finasset_ret(r_m, r_s, market_[:, 0], rf) @ x) + (labor_income @ discount_rate).values) ** (1-g)) / (1-g))

    cons = [cp.sum(x) == 1, x >= 0]
    prob = cp.Problem(cp.Minimize(obj), cons)
    prob.solve(cp.CVXOPT)
    print(g, np.round(x.value, 3)*100)

In [ ]:
-cp.sum((((fa * gen_finasset_ret(r_m, r_s, market_[:, 0], rf) @ x) + (labor_income @ discount_rate).values) ** (1-g)) / (1-g))


In [ ]:
risk_coeff = (1-g)

In [ ]:
(cp.power((gen_finasset_ret(0.12, 0.33, market_[:, 0], rf) @ x + (labor_income @ discount_rate).values), risk_coeff) / risk_coeff).size

In [ ]:
np.sum(cp.power(gen_finasset_ret(0.12, 0.33, market_[:, 0], rf) @ cp.Variable([2,1]) + np.array([(labor_income @ discount_rate).values]).T, risk_coeff) / risk_coeff)

In [ ]:
(cp.power((gen_finasset_ret(0.12, 0.33, market_[:, 0], rf) @ x + (labor_income @ discount_rate).values), risk_coeff) / risk_coeff)

In [ ]:
cp.sum(cp.power(gen_finasset_ret(0.12, 0.33, market_[:, 0], rf) @ x + np.array([(labor_income @ discount_rate).values]).T, risk_coeff) / risk_coeff)

In [ ]:
gamma_list = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]
for g in gamma_list:
    x = cp.Variable(2)
    
    
    #print(cp.sum(cp.power((gen_finasset_ret(0.12, 0.33, market_[:, 0], rf) @ x + (labor_income @ discount_rate).values), risk_coeff) / risk_coeff).size)
    if g == 1:
        obj = -cp.sum(cp.log((fa * gen_finasset_ret(r_m, r_s, market_[:, 0], rf) @ x) + (labor_income @ discount_rate).values))
    else:
        obj = cp.sum(((gen_finasset_ret(0.12, 0.33, market_[:, 0], rf) @ x + np.array([(labor_income @ discount_rate).values]).T) ** risk_coeff) / risk_coeff)

    cons = [cp.sum(x) == 1, x >= 0]
    prob = cp.Problem(cp.Maximize(obj), cons)
    prob.solve(cp.CPLEX)
    print(g, np.round(x.value, 3)*100)

In [ ]:
x0 = np.array([0.5, 0.5])
x = cp.Variable(2)
obj = cp.sum((((fa * cp.bmat(gen_finasset_ret(r_m, r_s, market_[:, 0], rf)) @ x) + (labor_income @ discount_rate).values) ** (1-gamma)) / (1-gamma))

cons = [cp.sum(x) == 1, x >= 0]
prob = cp.Problem(cp.Maximize(obj), cons)
prob.solve(cp.CPLEX)
print(np.round(x.value, 3)*100)

In [ ]:
def gen_labor_income(initial_hc, mu_h, sigma_h, eta, hc_, rf, size=20000, period=30):
    hc = np.ones([size, ]) * initial_hc
    
    hc_list = []
    for i in range(period):
        hc_t = hc * np.exp(mu_h + (sigma_h * hc_[:, i]))
        hc_list.append(hc_t)
        hc = hc_t
        
    return pd.DataFrame(hc_list).T


def get_discount_rate(rf, eta, period=30):
    dc_rate = np.exp((rf + 0.04 + eta) * (np.array(range(period)) + 1) * (-1))
    return dc_rate

# human capital = gen_labor_income @ dc_rate


def gen_finasset_ret(mu_market, sigma_market, market_, rf):
    risky_asset_ret = np.exp(mu_market - (0.5*(sigma_market**2)) + sigma_market*market_)
    rf_asset_ret = np.exp(rf * np.ones(market_.shape))
    
    return np.concatenate([[risky_asset_ret[:, 0]], [rf_asset_ret[:, 0]]]).T


def get_finasset_wealth(fin_asset_ret, initial_wealth, nomial_ic, consume_rate, risky_weights, rf_weights, size=20000):
    invest_amount = initial_wealth + (nomial_ic * (consume_rate))
    risky_asset = (invest_amount*risky_weights) * fin_asset_ret[0].cumprod(axis=1)
    rf_asset = (invest_amount*rf_weights) * fin_asset_ret[1].cumprod(axis=1)
    
    return risky_asset+rf_asset

In [ ]:
def get_maxutility_weights(W0, LI0, period, size, mu_rf, sigma_rf, mu_market, sigma_market, mu_h,
                           sigma_h, eta, rho, gamma):
    eta = eta
    def objective_fun(weights, W0, LI0, period, size, mu_rf, mu_market, sigma_market, mu_h,
                      sigma_h, eta, rho, gamma):
        np.random.seed(period)

        market_ = stats.norm.rvs(size=(size, period + 1), loc=0, scale=1)
        brownian_ = stats.norm.rvs(size=(size, period + 1), loc=0, scale=1)
        hc_ = rho * market_ + np.sqrt(1 - rho ** 2) * brownian_

        financial_ret = gen_finasset_ret(mu_market, sigma_market, market_, mu_rf)
        portfolio_ret = financial_ret @ weights
        
        W = (W0 + (LI0 * 0.3)) * portfolio_ret
        
        if period == 0:
            H = np.zeros(W.shape)
        else:
            LI = gen_labor_income(LI0, mu_h, sigma_h, eta, hc_, mu_rf, size, period)
            discounts_rate = get_discount_rate(mu_rf, eta, period)
            
            H = LI @ discounts_rate

        u_ = ((W + H) ** (1 - gamma)) / (1 - gamma)

        return -np.mean(u_)
    
    weights = np.array([0.5, 0.5])
    cons = ({'type': 'ineq', 'fun': lambda weight: 1.0 - np.sum(weights)},
            {'type': 'ineq', 'fun': lambda weight: -weights},)
    optimized = minimize(objective_fun, weights, (W0, LI0, period, size, mu_rf, mu_market, sigma_market, mu_h, 
                                                  sigma_h, eta, rho, gamma), method='COBYLA', constraints=cons)
    if not optimized.success: raise BaseException(optimized.message)
    return optimized.x

In [ ]:
size = 20000
period = 30
gamma = 2
rho = 0.2

ic = 5000
fa = 1000

hc_mu = 0.04
hcs = 0.01
eta = 0.0000002

r_m = 0.12
r_s = 0.33
rf = 0.04
rf_s = 0

In [ ]:
get_maxutility_weights(fa, ic, period, size, rf, rf_s, r_m, r_s, hc_mu, hcs, eta, rho, gamma)